In [97]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


In [98]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv() 
client = OpenAI()

In [99]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
loader = WebBaseLoader("https://www.metro.sp.gov.br/")
docs = loader.load()
    
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
splitDocs = splitter.split_documents(docs)

embedding = OpenAIEmbeddings()
vectorStore = FAISS.from_documents(docs, embedding=embedding)
retriever = vectorStore.as_retriever(search_kwargs={"k": 3})


In [100]:
agente_rotas_template = """Você é um especialista no sistema de Metrô de São Paulo, com foco em determinar as melhores rotas para os passageiros. \
      Você tem amplo conhecimento sobre as linhas do metrô, suas interligações e horários de pico, fornecendo alternativas rápidas e eficientes para quem utiliza o metrô.
"""

agente_horarios_template = """Você é um especialista em atualizações sobre o sistema de Metrô de São Paulo. \
    Você informa sobre greves, interrupções e atrasos, além de conhecer detalhadamente os horários de operação do metrô. \
        Você sugere opções alternativas para os passageiros durante períodos de interrupção do serviço.

"""

agente_politicas_template = """Você é um especialista em políticas públicas relacionadas ao sistema de Metrô de São Paulo. \
    Você tem conhecimento detalhado sobre as leis, regulamentos e iniciativas que afetam o metrô, e oferece respostas claras \
        e baseadas em fatos sobre como essas políticas impactam os passageiros.
"""



In [101]:
rotas_perguntas = [
    "Qual é a rota mais rápida da estação Sé até a estação Vila Madalena?",
    "Como posso evitar as estações mais lotadas durante o horário de pico?",
    "Quais são as melhores conexões entre as linhas azul e verde do metrô?",
]

horarios_perguntas = [
    "Há alguma greve no Metrô de São Paulo programada para esta semana?",
    "O metrô está funcionando normalmente durante o feriado?",
    "Quais são os horários de operação do metrô aos domingos?",
]

politicas_perguntas = [
    "Quais são as políticas de acessibilidade nas estações do Metrô de São Paulo?",
    "Como as tarifas do metrô são decididas e ajustadas?",
    "Quais são as políticas de sustentabilidade adotadas pelo Metrô de São Paulo?",
]


In [102]:
embeddings = OpenAIEmbeddings()

rotas_perguntas_embeddings = embeddings.embed_documents(rotas_perguntas)
horarios_perguntas_embeddings = embeddings.embed_documents(horarios_perguntas)
politicas_perguntas_embeddings = embeddings.embed_documents(politicas_perguntas)

In [103]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])

    rotas_similarity = cosine_similarity([query_embedding], rotas_perguntas_embeddings)[0]

    horarios_similarity = cosine_similarity([query_embedding], horarios_perguntas_embeddings)[0]

    politicas_similarity = cosine_similarity([query_embedding], politicas_perguntas_embeddings)[0]

    max_similarity = max(
        max(rotas_similarity), max(horarios_similarity), max(politicas_similarity)
    )

    if max_similarity == max(rotas_similarity):
        prompt_escolhido = "rotas"
    elif max_similarity == max(horarios_similarity):
        prompt_escolhido = "horarios"
    else:
        prompt_escolhido = "politicas"
    return prompt_escolhido
input_query = {"query": "Quando o metrô abre e fecha nos finds?"}
prompt = prompt_router(input_query)

In [104]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools.retriever import create_retriever_tool


search = DuckDuckGoSearchRun()
retriever_tools = create_retriever_tool(
    retriever,
    "metro_sp_search",
    "Use o campo de busca para encontrar informações sobre o Metrô de São Paulo.",
)
tools = [search, retriever_tools]

In [105]:
from langchain.agents import create_openai_functions_agent, AgentExecutor

model = ChatOpenAI(
    model='gpt-3.5-turbo-1106',
    temperature=0.7
)

In [106]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

def gerar_prompt(prompt_escolhido):


    prompt = ChatPromptTemplate.from_messages(
    [
        ("system", agente_rotas_template),

        ("user", "{input}"),

        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
    




    if prompt_escolhido == "rotas":

        prompt = ChatPromptTemplate.from_messages(
        [
            ("system", agente_horarios_template),

            ("user", "{input}"),

            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )


    else:
        prompt = ChatPromptTemplate.from_messages(
        [
            ("system", agente_politicas_template),

            ("user", "{input}"),

            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    
    return prompt



In [111]:
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
pergunta = "Como vou da fradique coutinho ate a sacoma?"
prompt = gerar_prompt(prompt_router({"query": pergunta}))

from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(model, tools, prompt)
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Você é um especialista em atualizações sobre o sistema de Metrô de São Paulo.     Você informa sobre greves, interrupções e atrasos, além de conhecer detalhadamente os horários de operação do metrô.         Você sugere opções alternativas para os passageiros durante períodos de interrupção do serviço.\n\n')), HumanMessag

In [112]:


agent_executer.invoke({"input":"Como vou da fradique coutinho ate a sacoma?"})





> Entering new AgentExecutor chain...

Invoking: `metro_sp_search` with `{'query': 'fradique coutinho para sacoma'}`












Metrô | Cia. do Metropolitano de São Paulo





























 





Skip to main content











SP + Digital







/governosp













SP + Digital







/governosp







 




















Canais de Relacionamento
 








Sua Viagem Mapa da Rede
Linhas e Estações
Acessibilidade e Atendimento Preferencial
Veja mais…


Expansão e Obras Andamentos das Obras
Canal de Relacionamento
Linha 2-Verde
Linha 15-Prata
Linha 17-Ouro
Veja mais…


Negócios
Governança
Transparência
Institucional
Investidores
 
















Para onde você vai?Saiba como chegar ao seu destino






Direto do Metrô






Declaração de Ocorrências
Foi afetado por alguma ocorrência nas linhas operadas pelo Metrô?
        Emita sua declaração.


Ver declarações disponíveis




Canais de Relacionamento

SMS Segurança
11 97333 2252

Informe a ocorrência de comércio i

{'input': 'Como vou da fradique coutinho ate a sacoma?',
 'output': 'Parece que não estou conseguindo encontrar os horários do metrô de Fradique Coutinho para Sacomã. No entanto, geralmente, o metrô de São Paulo opera das 4h40 à meia-noite, de domingo a sexta-feira, e das 4h40 à 1h aos sábados. Durante esse horário, você pode pegar a Linha 4-Amarela do metrô de Fradique Coutinho até a estação Paulista e, em seguida, transferir para a Linha 2-Verde em direção à estação Vila Prudente. De lá, você pode pegar um ônibus ou outro meio de transporte para chegar a Sacomã. Lamento a falta de informações mais precisas, mas espero que isso ajude!'}

In [109]:
"""
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)
"""

'\nchain = (\n    {"query": RunnablePassthrough()}\n    | RunnableLambda(prompt_router)\n    | ChatOpenAI()\n    | StrOutputParser()\n)\n'

In [110]:
"chain.invoke("Como vou da fradique coutinho ate a sacoma?")"

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1616276916.py, line 1)